# 📊 접근 로그 분석 노트북

**분석 대상**: `suwon_portal` (수원시 내부 포털 접근 로그)

---

## 1. 환경 설정

In [2]:
# ===== Autoreload 설정 =====
%load_ext autoreload
%autoreload 2

import os

import matplotlib as mpl

# ===== 시각화 설정 (Robust Font Fix) =====
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np

# ===== 기본 라이브러리 =====
import pandas as pd
import seaborn as sns

# ===== DB 유틸리티 =====
from db_utils import get_dataframe, get_tables, list_databases, preview, save_dataframe

font_path = "/usr/share/fonts/noto/NotoSansCJKkr-Regular.otf"
if os.path.exists(font_path):
    # 폰트 등록 (Matplotlib 3.2+ 호환 방식)
    fm.fontManager.addfont(font_path)
    prop = fm.FontProperties(fname=font_path)
    plt.rcParams["font.family"] = prop.get_name()
    print(f"Font successfully set to: {prop.get_name()}")
else:
    print("Warning: Font file not found. Falling back to default.")

plt.rcParams["axes.unicode_minus"] = False
sns.set_theme(style="whitegrid", font=plt.rcParams["font.family"])

# ===== pandas 출력 설정 =====
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)

ModuleNotFoundError: No module named 'db_utils'

In [ ]:
# 사용 가능한 DB 목록 확인
list_databases()
# ========================================
# 🔧 이 셀만 수정하면 다른 DB로 전환 가능
# ========================================
DB_ALIAS = "suwon_platform"  # db_config.json에 정의된 DB 별칭

NameError: name 'list_databases' is not defined

## 2. 데이터 로드 및 전처리

In [ ]:
# 테이블 목록 확인
tables = get_tables(DB_ALIAS)
tables.head()

/DATA/jupyter_WorkingDirectory/notebook/Access_Analysis/db_utils.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,table_name,table_type
0,A_기후,BASE TABLE
1,A_위치,BASE TABLE
2,A_토지,BASE TABLE
3,access_record,BASE TABLE
4,access_record_prod,BASE TABLE


In [ ]:
# ===== 데이터 로드 및 운영자/일반 사용자 분리 =====
acc_query = """
            SELECT *
            FROM public.access_record_prod
            """

rss_query = """
            SELECT *
            FROM public.resource_prod
            """

acc_log = get_dataframe(DB_ALIAS, acc_query)
rss = get_dataframe(DB_ALIAS, rss_query)

# ===== 운영자 및 일반 사용자 분리 (Data Cleansing) =====
TOTAL_USERS = 5000
OPERATOR_ID = "pebble"

# 1. 운영자 로그 분리
op_log = acc_log[acc_log["user_id"] == OPERATOR_ID].copy()

# 2. 일반 사용자 로그 분리 (운영자 및 인턴 계정 제외)
# 'pebble' 제외 및 'intern' 포함 계정 제외 (대소문자 무시)
is_intern = acc_log["user_id"].str.contains("intern", case=False, na=False)
is_operator = acc_log["user_id"] == OPERATOR_ID

user_log = acc_log[~(is_operator | is_intern)].copy()
intern_log = acc_log[is_intern & ~is_operator].copy()

# 기본 분석 대상 설정 (일반 사용자 위주)
df = user_log

print(f"전체 로그: {acc_log.shape[0]:,}행")
print(
    f"일반 사용자 로그: {user_log.shape[0]:,}행 (고유 {user_log['user_id'].nunique():,}명)"
)
print(f"운영자({OPERATOR_ID}) 로그: {op_log.shape[0]:,}행")
print(
    f"인턴 계정 로그: {intern_log.shape[0]:,}행 (고유 {intern_log['user_id'].nunique():,}명) -> 분석 제외"
)

/DATA/jupyter_WorkingDirectory/notebook/Access_Analysis/db_utils.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


전체 로그: 1,331,224행
일반 사용자 로그: 684,787행 (고유 779명)
운영자(pebble) 로그: 508,052행
인턴 계정 로그: 138,385행 (고유 14명) -> 분석 제외


## 3. 데이터 탐색 (EDA)

In [ ]:
# 데이터 타입 및 결측치 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 684787 entries, 30 to 1331223
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   sequence   684787 non-null  int64         
 1   user_id    684787 non-null  object        
 2   page_url   684787 non-null  object        
 3   page_name  684787 non-null  object        
 4   ip_info    684787 non-null  object        
 5   flag       684787 non-null  int64         
 6   issuer     684787 non-null  object        
 7   issued     684787 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 47.0+ MB


In [ ]:
# 여기에 분석 코드 작성
df["page_name"].value_counts()

page_name
                         301770
지표 조회                    222243
데이터셋 등록                   73434
분류 항목 등록                  28890
지표조회                      28875
데이터셋 신규등록                 10761
빅데이터 분석                    5614
로그인                         939
위경도 변환                      914
Role-사용자 매핑 설정              891
카테고리신청                      769
Role-메뉴설정                   683
지표외부연동-DB                   636
로그아웃                        635
지표 등록 - 수동                  632
Role-메뉴 매핑 설정               494
데이터 맵                       458
공지사항                        417
지표외부연동-API                  412
사용자 현황                      389
보고서 URL 관리                  333
메뉴 관리                       312
사전 목록                       308
조직 목록                       302
지표 수동등록                     296
메뉴관리                        296
데이터관리                       283
임시사용자 관리                    267
리소스 사용률                     250
표준어사전                       244
사용자 목록                      23

In [ ]:
df[
    (df["page_url"].str.contains("/dataservice/detail/"))
    # &(df['page_name']=='데이터셋')
]

,sequence,user_id,page_url,page_name,ip_info,flag,issuer,issued
30,1331612,ch1177,/dataservice/detail/dataset,,105.1.84.231,0,system,2025-12-31 16:03:19.126
31,1331611,ch1177,/dataservice/detail/ad710ce0-888d-42ea-9648-c0...,,105.1.84.231,0,system,2025-12-31 16:03:19.108
35,1331607,ch1177,/dataservice/detail/ad710ce0-888d-42ea-9648-c0...,,105.1.84.231,0,system,2025-12-31 16:03:12.747
37,1331605,ch1177,/dataservice/detail/dataset,,105.1.84.231,0,system,2025-12-31 16:03:05.019
38,1331604,ch1177,/dataservice/detail/dataset,,105.1.84.231,0,system,2025-12-31 16:03:00.275
...,...,...,...,...,...,...,...,...
1324810,6415,1002265,/dataservice/detail/459e9c33-8c77-4e81-aedc-99...,,::1,0,system,2020-01-14 14:48:11.438
1324816,6409,1002265,/dataservice/detail/459e9c33-8c77-4e81-aedc-99...,,::1,0,system,2020-01-14 14:47:24.240
1324866,6359,1002265,/dataservice/detail/9ed5eb19-a8c0-412c-97c8-b1...,,::1,0,system,2020-01-14 14:40:15.320
1324872,6353,1002265,/dataservice/detail/9ed5eb19-a8c0-412c-97c8-b1...,,::1,0,system,2020-01-14 14:40:08.459


In [ ]:
rss.head()

,identifier,type,contact_point,keyword,landing_page,qualified_relation,theme,conforms_to,is_referenced_by,creator,description,issued,language,license,modified,publisher,relation,title,has_policy,qualified_attribution,extension,access_rights,rights
0,5b4ca4b8-7dcf-4fd4-a4a5-777577784812,dataset,56100300123,,http://105.1.2.177:28081/dataservice/detail?id...,,23b71c31-7961-416f-9672-7d5d26866e21,pebble,-,pebble,도시교통정보센터 교통 돌발 정보\nhttps://www.utic.go.kr/guid...,2025-12-31 23:01:13.248,KR,Creative Commons,NaT,pebble,None,전국 교통 돌발 정보 2025년 12월 31일 22시 - 1,None,56100300123,,public,Copyright 2025. Suwon City. all rights reserved
1,7e0ee779-702e-4534-8526-ffba9264b9bf,dataset,10144,,http://192.168.102.167:8080/dataservice/ec39b4...,,92e5d5b6-73d2-4f79-bbab-f6672ee73d2f,10144,heeyaa38,undefined,,2025-12-31 23:01:03.485,KR,Creative Commons,NaT,10144,None,환경 미세 먼지 20251231230100 - 1,None,,,public,Copyright 2025. Suwon City. all rights reserved
2,4d96327c-0c07-4f8f-a081-53ef38710c8d,dataset,56100300123,,http://105.1.2.177:28081/dataservice/detail?id...,,0863e892-d327-4372-bda8-2d8e65c02c79,pebble,-,pebble,빅데이터 포털 재난 안전용 지표\n빅데이터팀 내부적으로 받아온 재난 안전 API...,2025-12-31 23:01:03.417,KR,Creative Commons,NaT,pebble,None,수원시 소방 화재 정보 2025년 12월 31일 22시 - 1,None,56100300123,,public,Copyright 2025. Suwon City. all rights reserved
3,6a3777f7-57a6-435b-b714-963c8f46d9f7,dataset,56100300123,,http://105.1.2.177:28081/dataservice/detail?id...,,23b71c31-7961-416f-9672-7d5d26866e21,pebble,-,pebble,http://105.1.2.177:28081/dataservice/detail?id...,2025-12-31 23:01:02.714,KR,Creative Commons,NaT,pebble,None,수원시 교량 현황 표준_2 20251231230100 - 1,None,56100300123,,public,Copyright 2025. Suwon City. all rights reserved
4,91d1f478-078c-48ed-896e-176f377f76ce,dataset,56100300123,,http://105.1.2.177:28081/dataservice/detail?id...,,0863e892-d327-4372-bda8-2d8e65c02c79,pebble,-,pebble,빅데이터 포털 재난 안전용 지표\n빅데이터팀 내부적으로 받아온 재난 안전 API...,2025-12-31 23:01:02.443,KR,Creative Commons,NaT,pebble,None,수원시 소방 긴급 구조 정보 실시간 2025년 12월 31일 22시 - 1,None,56100300123,,public,Copyright 2025. Suwon City. all rights reserved


In [ ]:
df[df["page_url"].str.contains("/dataservice/detail")]["page_url"].str.split("/").str[
    -1
].value_counts()

page_url
detail                                  66747
dataset                                 65787
60482079-d6b4-4e8c-bee3-abfcdd5461d1     2967
9c183aea-440c-4447-ad2c-9ab10f972bce     2875
e26dad41-4794-43ff-b759-d5ced1f2439e     2462
                                        ...  
12e45b98-5164-46bf-a159-789b70e0436f        1
31c2d1fe-e033-4544-8899-300039b72971        1
7a84bbfb-6780-4a74-a27a-5401710eb43c        1
5f5d3a38-abe3-4a3c-92fe-fe00b9e196a4        1
e7481b57-d124-4d55-a865-c59a8fcce180        1
Name: count, Length: 1880, dtype: int64

## 4. 활성 사용자 분석 (Active User Analysis)

전체 기간에 걸친 DAU, WAU, MAU 추세를 분석하여 서비스의 성장성과 견고함을 확인합니다.

In [ ]:
# 활성 지표 계산 (DAU, WAU, MAU)
df['date'] = df['issued'].dt.date
df['week_start'] = df['issued'].dt.to_period('W').apply(lambda r: r.start_time)
df['month_start'] = df['issued'].dt.to_period('M').apply(lambda r: r.start_time)

# 사용자 통계
dau = df.groupby('date')['user_id'].nunique()
wau = df.groupby('week_start')['user_id'].nunique()
mau = df.groupby('month_start')['user_id'].nunique()

print(f"--- 일반 사용자 (User) 통계 ---")
print(f"평균 DAU: {dau.mean():.1f}명")
print(f"평균 WAU: {wau.mean():.1f}명")
print(f"평균 MAU: {mau.mean():.1f}명")
print(f"최근 MAU: {mau.iloc[-1]:.1f}명")
print(f"고착도 (Stickiness, DAU/MAU): {dau.mean()/mau.mean()*100:.2f}%")

In [ ]:
# 3단 추세 시각화 (DAU, WAU, MAU) + 추세선 추가
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=False)
plt.subplots_adjust(hspace=0.4)

# 1. DAU (7일 이동 평균 추가)
axes[0].plot(dau.index, dau.values, label="Daily Users", color="dodgerblue", alpha=0.3, linewidth=1)
axes[0].plot(dau.index, dau.rolling(window=7, min_periods=1).mean(), 
            label="7-Day Trend", color="blue", linewidth=2)
if not op_log.empty:
    op_active_days = op_log["issued"].dt.date.unique()
    axes[0].scatter(op_active_days, [dau.max() * 0.1] * len(op_active_days), 
                color="red", marker="|", label="Operator Active", alpha=0.5)
axes[0].set_title("Daily Active Users (DAU)", fontsize=13)
axes[0].set_ylabel("Users")
axes[0].legend(loc="upper left")
axes[0].grid(True, linestyle="--", alpha=0.5)

# 2. WAU (4주 이동 평균 추가)
axes[1].plot(wau.index, wau.values, label="Weekly Users", color="forestgreen", marker='o', markersize=4, alpha=0.4)
axes[1].plot(wau.index, wau.rolling(window=4, min_periods=1).mean(), 
            label="4-Week Trend", color="darkgreen", linewidth=2)
axes[1].set_title("Weekly Active Users (WAU)", fontsize=13)
axes[1].set_ylabel("Users")
axes[1].legend(loc="upper left")
axes[1].grid(True, linestyle="--", alpha=0.5)

# 3. MAU (선형 추세선 또는 단순 추세 표시)
axes[2].plot(mau.index, mau.values, label="Monthly Users", color="darkorange", marker='s', alpha=0.5)
# MAU는 데이터 포인트가 적으므로 전체 추세를 보기 위해 더 넓은 윈도우 사용
axes[2].plot(mau.index, mau.rolling(window=3, min_periods=1).mean(), 
            label="Quarterly Trend", color="red", linewidth=2, linestyle='--')
axes[2].set_title("Monthly Active Users (MAU)", fontsize=13)
axes[2].set_ylabel("Users")
axes[2].legend(loc="upper left")
axes[2].grid(True, linestyle="--", alpha=0.5)

plt.suptitle("수원시 내부 포털 활성 추세 분석 (w/ Trendlines)", fontsize=16, y=0.95)
plt.show()

### 4.2 분석 결과 및 인사이트 (활성 사용자)

- **성장성**: (차트를 확인하여 DAU/WAU/MAU의 우상향 여부 기록)
- **안정성**: WAU와 MAU의 변동폭을 통해 유저 유입의 안정성 확인
- **특이점**: DAU가 급증하거나 급감한 특정 일자의 이벤트 역추적 필요

## 5. 리텐션 분석 고도화 (Yearly-Monthly Cohort)

주간 단위보다 거시적인 트렌드를 파악하기 위해 연도별/월간 코호트 리텐션을 분석합니다.

In [ ]:
# 연도-월별 코호트 리텐션 시각화
def plot_monthly_retention(df):
    df = df.copy()
    df['month_start'] = df['issued'].dt.to_period('M').apply(lambda r: r.start_time)
    df['first_month'] = df.groupby('user_id')['month_start'].transform('min')
    df['cohort_month_idx'] = ((df['month_start'].dt.year - df['first_month'].dt.year) * 12 + 
                              (df['month_start'].dt.month - df['first_month'].dt.month))
    
    df['year'] = df['first_month'].dt.year.astype(str)
    years = sorted(df['year'].unique())
    
    for y in years:
        y_df = df[df['year'] == y]
        cohort_pivot = y_df.pivot_table(index='first_month', columns='cohort_month_idx', values='user_id', aggfunc='nunique')
        cohort_size = cohort_pivot.iloc[:, 0]
        retention_matrix = cohort_pivot.divide(cohort_size, axis=0) * 100
        
        plt.figure(figsize=(14, min(len(retention_matrix) * 0.5 + 2, 8)))
        sns.heatmap(retention_matrix, annot=True, fmt=".1f", cmap="YlGnBu", cbar_kws={'label': 'Retention (%)'})
        plt.title(f"{y}년 가입자 월간 코호트 리텐션 (%)", fontsize=14)
        plt.xlabel("Months After First Visit")
        plt.ylabel("Cohort Month")
        plt.show()

plot_monthly_retention(df)

### 5.2 리텐션 인사이트

- **연도별 차이**: 특정 연도 가입자의 유지력이 다른 연도 대비 높은지 확인
- **Dead Zone**: 리텐션이 급격히 떨어지는 시점(보통 1~2개월 차)을 파악하여 대응책 마련

## 6. 주요 기능 사용성 및 퍼널 분석 (Usage & Funnel)

사용자들이 가장 많이 사용하는 기능을 파악하고, 핵심 가치에 도달하는 과정을 정의합니다.

In [ ]:
# 상위 사용 페이지/기능 분석
top_pages = df['page_name'].replace('', 'Unknown').value_counts().head(15)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_pages.values, y=top_pages.index, palette='viridis')
plt.title("상위 15개 사용 기능 (Page View 기준)")
plt.xlabel("Page Views")
plt.ylabel("Function Name")
plt.show()

print("--- 주요 분석 대상 기능 후보 ---")
print(top_pages)

### 6.2 퍼널(Funnel) 정의 및 시각화

서비스의 핵심 흐름을 다음과 같이 정의하여 분석합니다.

1. **로그인(Entrance)**: 서비스 접속
2. **리소스 탐색(Search)**: 데이터셋 또는 지표 목록 열람
3. **지표 상세 조회(View)**: 개별 지표 클릭
4. **데이터 활용(Action)**: 빅데이터 분석 또는 다운로드

In [ ]:
# 퍼널 분석 데이터 구성
def calculate_funnel(df):
    funnel_steps = [
        ('1. Login', df['user_id'].nunique()),
        ('2. Exploration', df[df['page_name'].str.contains('목록|리스트|검색', na=False)]['user_id'].nunique()),
        ('3. Detail View', df[df['page_name'].str.contains('조회|상세', na=False)]['user_id'].nunique()),
        ('4. Advanced Analysis', df[df['page_name'].str.contains('빅데이터 분석|전환', na=False)]['user_id'].nunique())
    ]
    
    funnel_df = pd.DataFrame(funnel_steps, columns=['Step', 'UserCount'])
    funnel_df['Conversion'] = funnel_df['UserCount'] / funnel_df['UserCount'].values[0] * 100
    funnel_df['Step_Conversion'] = funnel_df['UserCount'].pct_change().apply(lambda x: (1+x)*100 if pd.notnull(x) else 100)
    return funnel_df

funnel_results = calculate_funnel(df)

# 시각화
plt.figure(figsize=(10, 6))
bars = plt.barh(funnel_results['Step'][::-1], funnel_results['UserCount'][::-1], color='skyblue')
plt.title("전체 서비스 이용 퍼널 (Unique Users)", fontsize=15)

# 값 표시
for i, v in enumerate(funnel_results['UserCount'][::-1]):
    plt.text(v + 5, i, f"{v}명 ({funnel_results['Conversion'][::-1].iloc[i]:.1f}%)", va='center')

plt.xlabel("Unique Users")
plt.show()

display(funnel_results)

### 6.3 퍼널 분석 결과 및 인사이트

- **최대 이탈 구간**: 어느 단계에서 가장 많은 유저가 이탈하는지 확인 (예: 2->3단계)
- **전략적 제안**: 발견된 병목 구간의 UX 개선 또는 기능 노출 강화 필요

## 7. 아하 모먼트 (Aha! Moment) 발굴

리텐션에 결정적인 영향을 미치는 행동 임계값(Threshold)을 발굴합니다.